In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import researchpy as rp
from scipy import stats

In [2]:
# Read in CSVs
dfc = pd.read_csv('CurrentAd_new - CurrentAd_new.csv')
dfa = pd.read_csv('GroupA - GroupA.csv')

In [3]:
# Preview dfc
dfc.head()

,index,uuid,num_achievements,num_exercises,num_points,Click,Group,Button,Banner
0,0,981943945,37,166,254,1,CurrentAd,No,Yes
1,2,981944745,93,308,381,0,CurrentAd,No,Yes
2,3,981944801,91,260,306,0,CurrentAd,No,Yes
3,4,981944857,15,424,435,0,CurrentAd,No,Yes
4,6,981960025,41,293,374,0,CurrentAd,No,Yes


In [4]:
# Preview dfa
dfa.head()

,index,uuid,num_achievements,num_exercises,num_points,Click,Group,Button,Banner
0,8,981949649,0,0,0,0,A,No,No
1,15,981972297,0,0,0,0,A,No,No
2,22,981992089,0,0,0,0,A,No,No
3,32,981974489,0,0,0,0,A,No,No
4,38,981983905,0,0,0,0,A,No,No


In [5]:
# Combine the two tables into one since they contain all the same columns
df = pd.concat([dfc,dfa])

# Reset the index so each index value is unique
df.reset_index(drop = True, inplace = True)

# Drop index column because it isn't sequencial in favor of using the new index just created
df.drop(columns = 'index', inplace = True)

In [6]:
# View table to inspect results
df

,uuid,num_achievements,num_exercises,num_points,Click,Group,Button,Banner
0,981943945,37,166,254,1,CurrentAd,No,Yes
1,981944745,93,308,381,0,CurrentAd,No,Yes
2,981944801,91,260,306,0,CurrentAd,No,Yes
3,981944857,15,424,435,0,CurrentAd,No,Yes
4,981960025,41,293,374,0,CurrentAd,No,Yes
5,981954313,71,341,140,1,CurrentAd,No,Yes
6,981949961,60,547,18,0,CurrentAd,No,Yes
7,981952689,9,411,12,0,CurrentAd,No,Yes
8,982000809,65,62,247,0,CurrentAd,No,Yes
9,981990857,72,380,496,1,CurrentAd,No,Yes


In [7]:
# Check data types of each column in the table
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27268 entries, 0 to 27267
Data columns (total 8 columns):
uuid                27268 non-null int64
num_achievements    27268 non-null int64
num_exercises       27268 non-null int64
num_points          27268 non-null int64
Click               27268 non-null int64
Group               27268 non-null object
Button              27268 non-null object
Banner              27268 non-null object
dtypes: int64(5), object(3)
memory usage: 1.7+ MB


In [8]:
# Make a list of all columns in table that will be set as category data types
category_columns = 'Group Button Banner'.split()

# For loop for assiging the above listed columns as category data types
for i in category_columns:
    df[i] = df[i].astype('category')

In [9]:
# Summary of the two ad campaigns and how many clicks each got
rp.summary_cat(df[['Group', 'Click']])

,Variable,Outcome,Count,Percent
0,Group,CurrentAd,14125,51.80
1,,A,13143,48.20
2,Click,0,26337,96.59
3,,1,931,3.41


In [10]:
# Construct crosstab table in prep for Chi-squared test
crosstab = pd.crosstab(df['Group'], df['Click'])
crosstab

Click,0,1
Group,,
CurrentAd,13727,398
A,12610,533


In [11]:
# Run Chi-squared test and store outputs
table, results = rp.crosstab(df['Click'], df['Group'], prop = 'col', test = 'chi-square')

# Return table with no click (0) or click (1) as a percentage of each group and overall
table

Group                  
            A CurrentAd     All
Click                          
0       95.94     97.18   96.59
1        4.06      2.82    3.41
All    100.00    100.00  100.00

In [12]:
# Return results of Chi-squared test 
results

# The results we care about in the table below are p-value and phi coefficient (in this table Cramer's phi)
# The p-value tells us that there is a statistically significant relationship between ad groups and clicks
# But the phi coefficient tells us that it is not a strong relationship

,Chi-square test,results
0,Pearson Chi-square ( 1.0) =,31.6261
1,p-value =,0.0000
2,Cramer's phi =,0.0341


In [13]:
# Next I am taking a look at the two different user groups that were introducced to ad A and CurrentAd 
# and what we know about them

# Descriprive statistics for ad A
df[df['Group'] == 'A'].describe()

,uuid,num_achievements,num_exercises,num_points,Click
count,1.314300e+04,13143.000000,13143.000000,13143.000000,13143.000000
mean,1.034984e+09,1.029217,4.082553,4.082553,0.040554
std,6.657932e+07,0.894903,4.025102,4.025102,0.197262
min,9.616267e+08,0.000000,0.000000,0.000000,0.000000
25%,9.821693e+08,0.000000,0.000000,0.000000,0.000000
50%,9.839240e+08,1.000000,2.000000,2.000000,0.000000
75%,1.099113e+09,2.000000,8.000000,8.000000,0.000000
max,1.118685e+09,2.000000,18.000000,18.000000,1.000000


In [14]:
# Descriptive statistics for CurrentAd
df[df['Group'] == 'CurrentAd'].describe()

# It is worth noting that The two user groups are completely different by num_achievements, num_exercises,
# and num_points.

,uuid,num_achievements,num_exercises,num_points,Click
count,1.412500e+04,14125.00000,14125.000000,14125.000000,14125.000000
mean,1.044499e+09,49.53600,346.300956,352.137204,0.028177
std,6.771463e+07,28.74235,201.306499,199.499870,0.165484
min,9.616265e+08,0.00000,0.000000,0.000000,0.000000
25%,9.820278e+08,25.00000,174.000000,181.000000,0.000000
50%,1.097832e+09,50.00000,346.000000,353.000000,0.000000
75%,1.099048e+09,74.00000,519.000000,525.000000,0.000000
max,1.118159e+09,99.00000,699.000000,699.000000,1.000000
